In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
# โหลดข้อมูลจากไฟล์ CSV
df = pd.read_csv("../Dataset/Fraud.csv")

In [7]:
# ลบคอลัมน์ที่ไม่จำเป็น
df = df.drop(["nameOrig", "nameDest", "isFlaggedFraud"], axis=1)

In [8]:
# แปลงข้อมูลประเภท 'type' ให้เป็นตัวเลข
encoder = LabelEncoder()
df['type'] = encoder.fit_transform(df['type'])

In [9]:
# การลบ outliers โดยใช้ IQR
numerical_cols = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)  # คำนวณ Q1
    Q3 = df[col].quantile(0.75)  # คำนวณ Q3
    IQR = Q3 - Q1  # คำนวณ IQR
    lower_bound = Q1 - 1.5 * IQR  # หาขอบล่าง
    upper_bound = Q3 + 1.5 * IQR  # หาขอบบน
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]  # กรองข้อมูลที่อยู่ในช่วงที่กำหนด

In [10]:
# แยก features และ target
X = df.drop("isFraud", axis=1)  # features (ลบคอลัมน์ 'isFraud' ออก)
y = df["isFraud"]  # target (คอลัมน์ 'isFraud' คือป้ายกำกับที่บ่งบอกว่าข้อมูลนั้นเป็นการทุจริตหรือไม่)

In [11]:
# Normalization ของข้อมูล
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # ปรับขนาดข้อมูลให้มีค่าเฉลี่ยเป็น 0 และส่วนเบี่ยงเบนมาตรฐานเป็น 1

In [12]:
# แบ่งข้อมูลเป็นชุดฝึกอบรมและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
# สร้างโมเดลNeural Network
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),  # ชั้นที่ 1 มี 16 นิวรอน ใช้ฟังก์ชัน activation แบบ relu
    layers.Dense(8, activation='relu'),  # ชั้นที่ 2 มี 8 นิวรอน ใช้ฟังก์ชัน activation แบบ relu
    layers.Dense(1, activation='sigmoid')  # ชั้นสุดท้าย 1 นิวรอน ใช้ฟังก์ชัน activation แบบ sigmoid
])

C:\Users\mostm\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# ฝึกโมเดล
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 147s 2ms/step - accuracy: 0.9984 - loss: 0.0091 - val_accuracy: 0.9993 - val_loss: 0.0024
Epoch 2/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 150s 2ms/step - accuracy: 0.9995 - loss: 0.0019 - val_accuracy: 0.9994 - val_loss: 0.0022
Epoch 3/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 146s 2ms/step - accuracy: 0.9995 - loss: 0.0018 - val_accuracy: 0.9995 - val_loss: 0.0018
Epoch 4/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 128s 2ms/step - accuracy: 0.9995 - loss: 0.0017 - val_accuracy: 0.9995 - val_loss: 0.0020
Epoch 5/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 128s 2ms/step - accuracy: 0.9995 - loss: 0.0017 - val_accuracy: 0.9995 - val_loss: 0.0017
Epoch 6/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 127s 2ms/step - accuracy: 0.9995 - loss: 0.0016 - val_accuracy: 0.9995 - val_loss: 0.0016
Epoch 7/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 127s 2ms/step - accuracy: 0.9995 - loss: 0.0017 - val_accuracy: 0.9995 - val_loss: 0.0014
Epoch 8/10
81286/81286 ━━━━━━━━━━━━━━━━━━━━ 127s 2ms/step - ac

In [16]:
# ประเมินโมเดล
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")  # แสดงผลความแม่นยำจากชุดทดสอบ

20322/20322 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step - accuracy: 0.9995 - loss: 0.0016
Test Accuracy: 0.9995


In [18]:
# Save
model.save("../Model/fraud_detection_model.keras")  